In [6]:
#import pandas library
import pandas as pd

In [43]:
#read URL to datasets
cons_info = 'https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv'
cons_email = 'https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv'
subs_status = 'https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv'

In [44]:
#load the datasets
cons_info_df = pd.read_csv(cons_info)
cons_email_df = pd.read_csv(cons_email)
subs_status_df = pd.read_csv(subs_status)

In [45]:
#information of columns of each dataset to help understand which column from which dataset is needed for merging
cons_info_df.info()
cons_email_df.info()
subs_status_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 29 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   cons_id                     700000 non-null  int64  
 1   prefix                      350304 non-null  object 
 2   firstname                   350244 non-null  object 
 3   middlename                  560213 non-null  object 
 4   lastname                    349314 non-null  object 
 5   suffix                      349541 non-null  object 
 6   salutation                  350021 non-null  object 
 7   gender                      349891 non-null  object 
 8   birth_dt                    349954 non-null  object 
 9   title                       350082 non-null  object 
 10  employer                    349228 non-null  object 
 11  occupation                  350239 non-null  object 
 12  income                      350637 non-null  float64
 13  source        

In [46]:
# Filtering primary emails from the constituent email dataset
cons_email_df = cons_email_df[cons_email_df['is_primary'] == 1]
# Converting 'is_primary' to boolean
cons_email_df['is_primary'] = cons_email_df['is_primary'].astype(bool)
#see first few data rows of the filtered dataset
cons_email_df.head(5)
# Merge email data with constituent information
merged_df = cons_email_df.merge(cons_info_df[['cons_id', 'source']], on='cons_id', how='left')

<ipython-input-46-6d35cd8d37a6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cons_email_df['is_primary'] = cons_email_df['is_primary'].astype(bool)


In [47]:
# Filtering chapter_id 1 from the subscription status
subs_status_df = subs_status_df[subs_status_df['chapter_id'] == 1]
#see first few data rows of the filtered dataset
subs_status_df.head(5)

# Merge with subscription status; filter by chapter_id and handle missing 'isunsub'
subs_status_df = subs_status_df[subs_status_df['chapter_id'] == 1]
final_df = merged_df.merge(subs_status_df[['cons_email_id', 'isunsub']], on='cons_email_id', how='left')
final_df['is_unsub'] = final_df['isunsub'].fillna(0).astype(bool)  # Replace na values

In [48]:
# Select and rename the required columns for people.csv
people_df = final_df[['email', 'source', 'is_unsub', 'create_dt', 'modified_dt']]
people_df.columns = ['email', 'code', 'is_unsub', 'created_dt', 'updated_dt']

In [49]:
# Save to CSV
people_df.to_csv('people.csv', index=False)
people_df.head(5)

,email,code,is_unsub,created_dt,updated_dt
0,xmartinez@vincent.com,NaN,False,"Wed, 1994-01-26 23:49:16","Sat, 2014-04-19 19:10:39"
1,hmiller@haynes.biz,google,False,"Thu, 1999-12-09 06:18:27","Sat, 1984-07-14 05:55:27"
2,aaron64@yahoo.com,NaN,True,"Wed, 1992-11-18 16:46:27","Sun, 1995-12-24 13:13:01"
3,wyattvincent@hotmail.com,NaN,True,"Sat, 1983-11-26 16:49:14","Sun, 1975-01-19 14:32:56"
4,tspencer@hotmail.com,twitter,True,"Wed, 2000-11-15 13:28:34","Sun, 1994-03-13 16:38:37"


In [50]:
people_df['created_dt'] = pd.to_datetime(people_df['created_dt'])

# Extract date from 'created_dt'
people_df['acquisition_date'] = people_df['created_dt'].dt.date

# Group by 'acquisition_date' and count constituents
acquisition_facts_df = people_df.groupby('acquisition_date').size().reset_index(name='acquisitions')

# Save the the Acquisition dataframe to CSV
acquisition_facts_df.to_csv('acquisition_facts.csv', index=False)
acquisition_facts_df.head(5)

<ipython-input-50-c606de1644ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_df['created_dt'] = pd.to_datetime(people_df['created_dt'])
<ipython-input-50-c606de1644ef>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_df['acquisition_date'] = people_df['created_dt'].dt.date


,acquisition_date,acquisitions
0,1970-01-01,45
1,1970-01-02,29
2,1970-01-03,26
3,1970-01-04,31
4,1970-01-05,31
